In [36]:
import pandas as pd
import xarray as xr
import geopandas as gpd
import rasterio
import numpy as np
import richdem as rd
from rasterio.transform import rowcol
from datetime import datetime

# Load and prepare VIIRS data
gdf = gpd.read_file('custom_date_fires.geojson')

# Convert VIIRS data to DEM's CRS (EPSG:4269)
with rasterio.open('USGS3DEP_30m_33.5_34.5_-119.0_-118.0.tif') as src:
    dem_crs = src.crs
gdf = gdf.to_crs(dem_crs)

# Load and prepare ERA5 data
era5_wind = xr.open_dataset('era5_wind_la.nc', engine='netcdf4').rename({'valid_time': 'time'})

# Load DEM and calculate slope
with rasterio.open('USGS3DEP_30m_33.5_34.5_-119.0_-118.0.tif') as src:
    dem = src.read(1)
    transform = src.transform
    dem_bounds = src.bounds
    dem_height, dem_width = dem.shape

# Clip VIIRS points to DEM bounds
gdf_clipped = gdf.cx[
    dem_bounds.left:dem_bounds.right,
    dem_bounds.bottom:dem_bounds.top
]

# Calculate slope using richdem
dem_rd = rd.rdarray(dem, no_data=-9999)
slope = rd.TerrainAttribute(dem_rd, attrib='slope_degrees')

def get_wind_at_point(lat, lon, viirs_time):
    """Get wind components at specific location and time"""
    # Convert VIIRS time to numpy datetime
    era5_time = np.datetime64(viirs_time)
    
    return era5_wind.sel(
        latitude=lat,
        longitude=lon,
        time=era5_time,
        method='nearest'
    )

# Process features with error handling
features = []
for _, row in gdf_clipped.iterrows():
    try:
        # Get wind data
        wind = get_wind_at_point(row.geometry.y, row.geometry.x, row['acq_date'])
        
        # Convert coordinates to DEM grid indices
        x, y = row.geometry.x, row.geometry.y
        row_idx, col_idx = rowcol(transform, x, y)
        
        # Check array bounds
        if (0 <= row_idx < dem_height) and (0 <= col_idx < dem_width):
            elevation = dem[row_idx, col_idx]
            slope_value = slope[row_idx, col_idx]
        else:
            print(f"Point out of DEM bounds: ({x}, {y})")
            elevation = np.nan
            slope_value = np.nan
        
        features.append({
            'latitude': row.geometry.y,
            'longitude': row.geometry.x,
            'frp': row['frp'],
            'u10': wind.u10.item(),
            'v10': wind.v10.item(),
            'elevation': elevation,
            'slope': slope_value,
            'confidence': row['confidence']
        })
        
    except Exception as e:
        print(f"Skipping row due to error: {str(e)}")
        continue

features_df = pd.DataFrame(features)
print("Successfully processed features:")
print(features_df.head())
print(f"\nTotal features: {len(features_df)}")
print(f"NaN values:\n{features_df.isna().sum()}")


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)



A Slope calculation (degrees)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.235133======================= ] (99% - 0.0s - 1 threads)


Successfully processed features:
   latitude  longitude   frp       u10       v10   elevation      slope  \
0  33.81573 -118.23775  1.05  1.259857  3.371613    1.781858  69.389099   
1  34.15556 -118.19301  1.56  0.662201  0.849152  434.277283  80.075356   
2  34.29366 -118.80275  1.33  0.376068  0.480011  297.056061  56.920059   
3  34.33566 -118.51929  1.25  0.539154  0.854034  581.089966  73.065117   
4  34.42990 -118.64459  0.86  0.599701 -1.086395  350.062622  75.363266   

  confidence  
0          n  
1          n  
2          n  
3          n  
4          n  

Total features: 1731
NaN values:
latitude      0
longitude     0
frp           0
u10           0
v10           0
elevation     0
slope         0
confidence    0
dtype: int64


In [37]:
import folium
from folium.plugins import HeatMap, MarkerCluster
import branca.colormap as cm

# Create base map centered on LA
m = folium.Map(location=[34.05, -118.25], zoom_start=9, tiles='CartoDB dark_matter')

# Add Slope Layer (as raster image)
slope_colormap = cm.LinearColormap(
    ['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'],
    vmin=0, vmax=90,
    caption='Slope (degrees)'
)

# Save slope as temporary GeoTIFF for visualization
with rasterio.open('slope.tif', 'w', 
                   driver='GTiff',
                   height=slope.shape[0],
                   width=slope.shape[1],
                   count=1,
                   dtype=slope.dtype,
                   crs=dem_crs,
                   transform=transform) as dst:
    dst.write(slope, 1)

# Add slope overlay
folium.raster_layers.ImageOverlay(
    image='slope.tif',
    bounds=[[dem_bounds.bottom, dem_bounds.left], 
            [dem_bounds.top, dem_bounds.right]],
    colormap=slope_colormap,
    opacity=0.6,
    name='Slope'
).add_to(m)

# Add VIIRS Fire Points colored by FRP
frp_colormap = cm.LinearColormap(
    ['yellow', 'orange', 'red'],
    vmin=features_df['frp'].min(),
    vmax=features_df['frp'].max(),
    caption='FRP (MW)'
)

for idx, row in features_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color=frp_colormap(row['frp']),
        fill=True,
        popup=f"FRP: {row['frp']} MW<br>Confidence: {row['confidence']}"
    ).add_to(m)

# Add Wind Vectors as Arrows
for idx, row in features_df.iterrows():
    folium.PolyLine(
        locations=[
            [row['latitude'], row['longitude']],
            [row['latitude'] + row['v10']/100,  # Scale for visibility
             row['longitude'] + row['u10']/100]
        ],
        color='#00b3ff',
        weight=1.5,
        tooltip=f"Wind: {np.sqrt(row['u10']**2 + row['v10']**2):.1f} m/s"
    ).add_to(m)

# Add layer control and save
folium.LayerControl().add_to(m)
frp_colormap.add_to(m)
m.save('fire_analysis_map.html')


RUNNING MODEL

In [38]:
import pandas as pd
import xarray as xr
import geopandas as gpd
import rasterio
import numpy as np
import richdem as rd
import folium
from folium.plugins import HeatMap
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from rasterio.transform import rowcol
from datetime import datetime

In [39]:
# Load VIIRS Fire Data
gdf = gpd.read_file('custom_date_fires.geojson')
print("VIIRS data columns:", gdf.columns.tolist())

# Convert to DEM CRS (EPSG:4269)
with rasterio.open('USGS3DEP_30m_33.5_34.5_-119.0_-118.0.tif') as src:
    dem_crs = src.crs
gdf = gdf.to_crs(dem_crs)
print("\nVIIRS CRS after conversion:", gdf.crs)

VIIRS data columns: ['country_id', 'latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_ti5', 'frp', 'daynight', 'geometry']

VIIRS CRS after conversion: GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4269"]]


In [40]:
# Load ERA5 Wind Data
era5_wind = xr.open_dataset('era5_wind_la.nc', engine='netcdf4').rename({'valid_time': 'time'})
print("\nERA5 variables:", list(era5_wind.data_vars))


# Load DEM and calculate slope
with rasterio.open('USGS3DEP_30m_33.5_34.5_-119.0_-118.0.tif') as src:
    dem = src.read(1)
    transform = src.transform
    dem_height, dem_width = dem.shape
    dem_bounds = src.bounds

# Calculate slope using richdem
dem_rd = rd.rdarray(dem, no_data=-9999)
slope = rd.TerrainAttribute(dem_rd, attrib='slope_degrees')
print("\nDEM and slope processed")

# Clip VIIRS points to DEM bounds
gdf_clipped = gdf.cx[
    dem_bounds.left:dem_bounds.right,
    dem_bounds.bottom:dem_bounds.top
]

# Process features with full validation
features = []
for idx, row in gdf_clipped.iterrows():
    try:
        # --- Wind Data Extraction ---
        # Convert VIIRS time to numpy datetime
        era5_time = np.datetime64(row['acq_date'])
        
        # Get nearest wind data
        wind = era5_wind.sel(
            time=era5_time,
            latitude=row.geometry.y,
            longitude=row.geometry.x,
            method='nearest'
        )
        
        # --- DEM/Slope Extraction ---
        # Convert coordinates to grid indices
        x, y = row.geometry.x, row.geometry.y
        row_idx, col_idx = rowcol(transform, x, y)
        
        # Validate indices
        valid_row = 0 <= row_idx < dem_height
        valid_col = 0 <= col_idx < dem_width
        
        if valid_row and valid_col:
            elevation = dem[row_idx, col_idx]
            slope_val = slope[row_idx, col_idx]
        else:
            elevation = np.nan
            slope_val = np.nan
        
        # --- Feature Collection ---
        features.append({
            'latitude': row.geometry.y,
            'longitude': row.geometry.x,
            'frp': row['frp'],
            'u10': wind.u10.item(),
            'v10': wind.v10.item(),
            'elevation': elevation,
            'slope': slope_val,
            'confidence': row['confidence'].lower()  # Ensure lowercase
        })
        
    except Exception as e:
        print(f"Skipping row {idx}: {str(e)}")
        continue

# Create DataFrame and clean data
features_df = pd.DataFrame(features).dropna(subset=['elevation', 'slope'])

# Convert confidence to numerical values (0=low, 1=nominal, 2=high)
#features_df['confidence_num'] = pd.factorize(features_df['confidence'])[0]
features_df['confidence_num'] = features_df['confidence'].map({'l':0, 'n':1, 'h':2})

print("\nProcessed features summary:")
print(f"Total valid features: {len(features_df)}")
print(f"Elevation range: {features_df['elevation'].min():.1f} - {features_df['elevation'].max():.1f} m")
print(f"Slope range: {features_df['slope'].min():.1f} - {features_df['slope'].max():.1f}°")
print("\nSample features:")
print(features_df.head())



ERA5 variables: ['u10', 'v10']
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)



A Slope calculation (degrees)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.274558======================= ] (99% - 0.0s - 1 threads)



DEM and slope processed

Processed features summary:
Total valid features: 1731
Elevation range: -0.3 - 1717.6 m
Slope range: 0.0 - 88.8°

Sample features:
   latitude  longitude   frp       u10       v10   elevation      slope  \
0  33.81573 -118.23775  1.05  1.259857  3.371613    1.781858  69.389099   
1  34.15556 -118.19301  1.56  0.662201  0.849152  434.277283  80.075356   
2  34.29366 -118.80275  1.33  0.376068  0.480011  297.056061  56.920059   
3  34.33566 -118.51929  1.25  0.539154  0.854034  581.089966  73.065117   
4  34.42990 -118.64459  0.86  0.599701 -1.086395  350.062622  75.363266   

  confidence  confidence_num  
0          n               1  
1          n               1  
2          n               1  
3          n               1  
4          n               1  


In [41]:
print("Unique confidence values: ",gdf_clipped['confidence'].unique())
print(features_df['confidence_num'].unique())

print("NaN values in confidence_num:", features_df['confidence_num'].isna().sum())


Unique confidence values:  ['n' 'l' 'h']
[1 0 2]
NaN values in confidence_num: 0


In [42]:
# ## Machine Learning Model
# %%
# Convert confidence to numerical values
#confidence_map = {'l': 0, 'n': 1, 'h': 2}
#features_df['confidence_num'] = features_df['confidence'].map(confidence_map)

# Prepare data
X = features_df[['frp', 'u10', 'v10', 'elevation', 'slope']]
y = features_df['confidence_num']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
score = model.score(X_test, y_test)
print(f"\nModel R² Score: {score:.2f}")

features_df['predicted_confidence'] = model.predict(X)
features_df['predicted_confidence_rounded'] = features_df['predicted_confidence'].round().astype(int)



Model R² Score: -0.15


In [34]:
# ## Interactive Visualization
# %%
# Create base map
m = folium.Map(location=[34.05, -118.25], zoom_start=9, tiles='CartoDB dark_matter')

# Add slope layer
slope_layer = folium.raster_layers.ImageOverlay(
    image=slope,
    bounds=[[dem_bounds.bottom, dem_bounds.left], 
            [dem_bounds.top, dem_bounds.right]],
    colormap=lambda x: (1, 0, 0, x/90),  # Red gradient
    opacity=0.4,
    name='Slope'
).add_to(m)

# Add fire points
for idx, row in features_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['frp']/5,
        color='#ff4500',
        fill=True,
        popup=f"FRP: {row['frp']} MW<br>Slope: {row['slope']:.1f}°"
    ).add_to(m)

# Add wind vectors
for idx, row in features_df.iterrows():
    folium.PolyLine(
        locations=[
            [row['latitude'], row['longitude']],
            [row['latitude'] + row['v10']/100, 
             row['longitude'] + row['u10']/100]
        ],
        color='#00b3ff',
        weight=1.5,
        tooltip=f"Wind Speed: {np.sqrt(row['u10']**2 + row['v10']**2):.1f} m/s"
    ).add_to(m)

folium.LayerControl().add_to(m)
m.save('wildfire_analysis.html')
print("\nInteractive map saved to wildfire_analysis.html")


TypeError: Object of type rdarray is not JSON serializable

In [ ]:
#DEBUGGING

In [1]:
import os
import geopandas as gpd

# Check file existence
viirs_path = 'custom_date_fires.geojson'
print(f"File exists: {os.path.exists(viirs_path)}")

# Load data with forced lowercase columns
gdf = gpd.read_file(viirs_path).rename(columns=lambda x: x.lower())
print("\nVIIRS columns:", gdf.columns.tolist())
print("\nUnique confidence values:", gdf['confidence'].unique())


File exists: True

VIIRS columns: ['country_id', 'latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_ti5', 'frp', 'daynight', 'geometry']

Unique confidence values: ['n' 'h' 'l']


In [4]:
# Convert VIIRS to DEM's CRS
with rasterio.open('USGS3DEP_30m_33.5_34.5_-119.0_-118.0.tif') as src:
    dem_crs = src.crs
    dem_bounds = src.bounds  # (-119.00027, 33.50027, -118.00027, 34.50027)

# Spatial filter: Keep only points within DEM bounds
gdf_clipped = gdf.cx[
    dem_bounds.left:dem_bounds.right,
    dem_bounds.bottom:dem_bounds.top
]

print(f"Original VIIRS points: {len(gdf)}")
print(f"Points within DEM bounds: {len(gdf_clipped)}")


Original VIIRS points: 5140
Points within DEM bounds: 1731


In [5]:
print("\nClipped VIIRS bounds:")
print(gdf_clipped.total_bounds)
print("DEM bounds:", dem_bounds)

# Sample output:
# Clipped VIIRS bounds: [-118.999, 33.501, -118.001, 34.499]
# DEM bounds: (-119.00027, 33.50027, -118.00027, 34.50027)



Clipped VIIRS bounds:
[-118.94359   33.77682 -118.01714   34.43543]
DEM bounds: BoundingBox(left=-119.00027728978193, bottom=33.50027737376589, right=-118.00027728178192, top=34.500277381765905)


In [43]:
import folium
import matplotlib.pyplot as plt
import numpy as np
import os

# Step 1: Normalize slope and save as image
slope_np = np.array(slope)
slope_norm = (slope_np - np.nanmin(slope_np)) / (np.nanmax(slope_np) - np.nanmin(slope_np))

slope_img_path = 'slope_overlay.png'
plt.imsave(slope_img_path, slope_norm, cmap='Reds')

# Step 2: Create base map
m = folium.Map(location=[34.05, -118.25], zoom_start=9, tiles='CartoDB dark_matter')

# Step 3: Add slope raster
folium.raster_layers.ImageOverlay(
    name='Slope',
    image=slope_img_path,
    bounds=[[dem_bounds.bottom, dem_bounds.left],
            [dem_bounds.top, dem_bounds.right]],
    opacity=0.4
).add_to(m)

# Step 4: Define color map
confidence_colors = {
    0: 'gray',    # low
    1: 'orange',  # nominal
    2: 'red'      # high
}

# Step 5: Create FeatureGroups for toggling
actual_layer = folium.FeatureGroup(name='Actual Confidence', show=True)
predicted_layer = folium.FeatureGroup(name='Predicted Confidence', show=False)

# Step 6: Add fire points — actual confidence
for idx, row in features_df.iterrows():
    actual_conf = row['confidence_num']
    color = confidence_colors.get(actual_conf, 'blue')
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['frp'] / 5 if row['frp'] > 0 else 2,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"Actual Confidence: {actual_conf}<br>FRP: {row['frp']} MW<br>Slope: {row['slope']:.1f}°"
    ).add_to(actual_layer)

# Step 7: Add fire points — predicted confidence
for idx, row in features_df.iterrows():
    predicted_conf = row['predicted_confidence_rounded']
    color = confidence_colors.get(predicted_conf, 'blue')
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['frp'] / 5 if row['frp'] > 0 else 2,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"Predicted Confidence: {predicted_conf}<br>FRP: {row['frp']} MW<br>Slope: {row['slope']:.1f}°"
    ).add_to(predicted_layer)

# Step 8: Add wind vectors to the base map
for idx, row in features_df.iterrows():
    folium.PolyLine(
        locations=[
            [row['latitude'], row['longitude']],
            [row['latitude'] + row['v10']/100, row['longitude'] + row['u10']/100]
        ],
        color='#00b3ff',
        weight=1.5,
        tooltip=f"Wind: {np.sqrt(row['u10']**2 + row['v10']**2):.1f} m/s"
    ).add_to(m)

# Step 9: Add the layers to the map
actual_layer.add_to(m)
predicted_layer.add_to(m)

# Step 10: Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Step 11: Save
m.save('wildfire_analysis_layers.html')
print("\nInteractive layered map saved to wildfire_analysis_layers.html")



Interactive layered map saved to wildfire_analysis_layers.html
